# Flappy Bird Junco Sim
2D flight sim for EECE5500 Mobile Robotics

In [177]:
############################### Import libraries ###############################
import os
import glob
import time
from datetime import datetime

import matplotlib.pyplot as plt

import numpy as np
from numpy import matlib

import gym
import control

[[2.41421356]]


In [300]:
class Drone():
    m = 3.2
    Iyy = 1/12*m*0.8**2

    S = 0.25
    c = 0.13

    Cl_0 = 0.5
    Cl_alpha = 0.17

    Cd_0 = 0.02
    K = 0.01

    Cm_0 = -0.05*0
    Cm_alpha = 0.4
    Cm_alpha_dot = -0.1*0
    Cm_delta_e = 0.5

    g = 9.81

    def __init__(self):
        self.x_thres = 1000
        self.z_thresHigh = 500
        self.z_thresLow = 0
        self.v_thres = 50
        self.theta_thres = np.pi/6
        self.theta_dot_thres = np.pi
        self.gamma_thres = np.pi/6
        self.alpha_dot_stale = 0

        self.limits = np.array([
            self.x_thres,
            self.z_thresHigh,
            self.z_thresLow,
            self.v_thres,
            self.theta_thres,
            self.theta_dot_thres,
            self.gamma_thres
        ])

        self.state = None
        self.prev_action = None
        self.steps_beyond_terminated = None

        self.time = 0.
        self.dt = 0.01

      
    def step(self, action):
        err_msg = f"{action!r} ({type(action)}) invalid"
        assert self.state is not None, "Call reset before using step method."
        
        self.time += self.dt

        #Extra states
        rho = np.random.normal(1.225,0.01)
        alpha = self.state[3] - self.state[4]
        self.q = 0.5*rho*self.state[2]**2
        
        #Control inputs
        thrust, delta_e = action

        # Forces
        Cl = self.Cl_0 + self.Cl_alpha*alpha
        Cd = self.Cd_0 + self.K*Cl**2
        Cm = self.Cm_0 + self.Cm_alpha*alpha + self.Cm_alpha_dot*self.alpha_dot_stale + self.Cm_delta_e*delta_e
        print(self.Cm_0, self.Cm_alpha*alpha, self.Cm_alpha_dot*self.alpha_dot_stale, self.Cm_delta_e*delta_e)
        L = self.q*self.S*Cl
        D = self.q*self.S*Cd
        M = self.q*self.S*Cm
        x, z, v, theta, theta_dot, gamma = self.state

        #derivatives
        x_dot = v*np.cos(gamma)
        z_dot = v*np.sin(gamma)
        v_dot = (-D - self.m*self.g*np.sin(gamma) + thrust*np.cos(alpha)) / self.m
        #theta_dot = theta_dot
        theta_ddot = M / self.Iyy
        gamma_dot = (L - self.m*self.g*np.cos(gamma) -thrust*np.sin(alpha)) / (self.m*v)

        #No algebraic loops in my flight code 
        self.alpha_dot_stale = theta_dot - gamma_dot

        # integrate
        x += x_dot*self.dt
        z += z_dot*self.dt
        v += v_dot*self.dt
        theta += theta_dot*self.dt #Removed second order terms.  We don't do that here (Also needs to be 1/2dt^2 f"(t))
        theta_dot += theta_ddot * self.dt
        gamma += gamma_dot*self.dt

        state = np.array([x,z,v,theta,theta_dot,gamma])
        action = np.array([thrust,delta_e])

        self.state = (list(np.reshape(state,(6,))))

        terminated = bool(
            x < -self.x_thres
            or x > self.x_thres
            or z > self.z_thresHigh
            or z < self.z_thresLow
            or v < -self.v_thres
            or v > self.v_thres
            or theta < -self.theta_thres
            or theta > self.theta_thres
            or theta_dot < -self.theta_dot_thres
            or theta_dot > self.theta_dot_thres
            or gamma < -self.gamma_thres
            or gamma > self.gamma_thres
        )

        if not terminated:
            reward = 1.0
        elif self.steps_beyond_terminated is None:
            self.steps_beyond_terminated = 0
            reward = 0.0
        else:
            if self.steps_beyond_terminated == 0:
                print(self.state)
                gym.logger.warn(
                    "You are calling 'step()' even though this "
                    "environment has already returned terminated = True. You "
                    "should always call 'reset()' once you receive 'terminated = "
                    "True' -- any further steps are undefined behavior."
                )
            self.steps_beyond_terminated += 1
            reward = 0.0

        return state, reward, terminated, False

    def reset(self):
        self.state = np.array([0,100,20,0,0,0])
        self.steps_beyond_terminated = None
        self.time = 0

        return self.state

    def close(self):
        pass

In [302]:
env = Drone()
state = env.reset()
action = np.array([1.5,0.000001])
fig, axs = plt.subplots(2, 2)
trajx = []
trajz = []
trajtheta = []
trajgamma = []
trajv = []

for t in range(1, 1000):
    state, reward, done, _ = env.step(action)
    x = state[0]
    z = state[1]
    v = state[2]
    theta = state[3]
    theta_dot = state[4]
    gamma = state[5]
    trajx.append(x)
    trajz.append(z)
    trajtheta.append(theta)
    trajgamma.append(gamma)
    trajv.append(v)

axs[0, 0].plot(trajx, trajz)
axs[0, 0].set_ylabel("Height (m)")
axs[0, 1].plot(trajx, trajtheta)
axs[0, 1].set_ylabel("Theta (rad)")
axs[1, 0].plot(trajx, trajgamma)
axs[1, 0].set_ylabel("Gamma (rad)")
axs[1, 1].plot(trajx, trajv)
axs[1, 1].set_ylabel("Velocity (m/s)")

fig.show()

-0.0 0.0 -0.0 5e-07
-0.0 -7.232303313074244e-07 -0.0 5e-07
-0.0 -3.9722415875658074e-07 -0.0 5e-07
-0.0 -5.412142531731145e-07 -0.0 5e-07
-0.0 -4.76013109982295e-07 -0.0 5e-07
-0.0 -5.052904167469669e-07 -0.0 5e-07
-0.0 -4.924449083716337e-07 -0.0 5e-07
-0.0 -4.980940634440037e-07 -0.0 5e-07
-0.0 -4.955167645914275e-07 -0.0 5e-07
-0.0 -4.966294308605107e-07 -0.0 5e-07
-0.0 -4.960488435051802e-07 -0.0 5e-07
-0.0 -4.962546203232244e-07 -0.0 5e-07
-0.0 -4.960398123953419e-07 -0.0 5e-07
-0.0 -4.961375030425736e-07 -0.0 5e-07
-0.0 -4.960375521478308e-07 -0.0 5e-07
-0.0 -4.960928491346699e-07 -0.0 5e-07
-0.0 -4.960751835648617e-07 -0.0 5e-07
-0.0 -4.958344091707386e-07 -0.0 5e-07
-0.0 -4.959026541207759e-07 -0.0 5e-07
-0.0 -4.959026059251689e-07 -0.0 5e-07
-0.0 -4.959048646104411e-07 -0.0 5e-07
-0.0 -4.957955802140035e-07 -0.0 5e-07
-0.0 -4.958047837849507e-07 -0.0 5e-07
-0.0 -4.956768086058223e-07 -0.0 5e-07
-0.0 -4.957598491098332e-07 -0.0 5e-07
-0.0 -4.956602314396323e-07 -0.0 5e-07
-0.0 

/tmp/ipykernel_41764/513938920.py:120: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  gym.logger.warn(


In [277]:
#################################### Testing ###################################

env = Drone()
state = env.reset()
action = np.array([1.5,1])

%matplotlib qt
fig, ax = plt.subplots(1,figsize=(10,5))
traj = []

for t in range(1, 1000):
    
    state, reward, done, _ = env.step(action)
    x = state[0]
    z = -state[1]
    v = state[2]
    theta = -state[3]
    theta_dot = -state[4]
    gamma = state[5]
    traj.append((x,z))

    action = np.array([0,0])
        
    if v<25:
        action[0] = 200
    else:
        action[0] = 0
        
    l = 0.8
    plt.plot([x,x+0.25*l*np.cos(theta)],[z,z+0.25*l*np.sin(theta)],'k-')
    plt.plot([x,x-0.75*l*np.cos(theta)],[z,z-0.75*l*np.sin(theta)],'k-')
    plt.xlim(x-20,x+80)
    plt.ylim(z-25,z+25)

    beam = 50
    angle = 60*np.pi/180
    plt.plot([x,x+beam*np.cos(theta+angle/2)],[z,z+beam*np.sin(theta+angle/2)],'k--')
    plt.plot([x,x+beam*np.cos(theta-angle/2)],[z,z+beam*np.sin(theta-angle/2)],'k--')
    xs = x+beam*np.cos(theta+np.linspace(-angle/2,angle/2,10))
    zs = z+beam*np.sin(theta+np.linspace(-angle/2,angle/2,10))
    plt.plot(xs,zs,'k.',markersize=1)

    x,z = zip(*traj)
    plt.plot(x,z,'b-',markersize=1)
    
    plt.show()
    plt.pause(env.dt)

    
    if done:
        break
    else:
        ax.clear()
print(np.around(state,2), reward, done)
env.close()

-0.05 0.0 -0.0 0.5
-0.05 -0.0009697229116953746 -0.0 0.0
-0.05 -0.000849600602594869 -0.0 0.0
-0.05 -0.0007231407435921738 0.0 0.0
-0.05 -0.000593235922048435 0.0 0.0
-0.05 -0.0004579998171103512 0.0 0.0
-0.05 -0.0003158134345572727 0.0 0.0
-0.05 -0.00016711074644369436 0.0 0.0
-0.05 -1.2440092930010078e-05 0.0 0.0
-0.05 0.00014753881004443724 0.0 0.0
-0.05 0.00031342373388791887 0.0 0.0
-0.05 0.00048587815009970786 0.0 0.0
-0.05 0.000660248791178296 0.0 0.0
-0.05 0.0008260530704258213 0.0 0.0
-0.05 0.0009897656758389409 0.0 0.0
-0.05 0.001151364319884539 0.0 0.0
-0.05 0.0013094744092825479 0.0 0.0
-0.05 0.0014677271648840817 0.0 0.0
-0.05 0.0016244993163092066 0.0 0.0
-0.05 0.0017729421277443386 0.0 0.0
-0.05 0.0019248744954004644 0.0 0.0
-0.05 0.0020738168336803774 0.0 0.0
-0.05 0.002221396676356883 0.0 0.0
-0.05 0.0023658594290971247 0.0 0.0
-0.05 0.0025058491299804046 0.0 0.0
-0.05 0.002643296232032212 0.0 0.0
-0.05 0.002779844868572727 0.0 0.0
-0.05 0.002913010563025907 0.0 0.0
-0